In [11]:
import pandas as pd
import geopandas as gpd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt
import  numpy as np
%matplotlib inline

In [33]:
labels = pd.read_csv('./input/vessels_labels_train.csv')
ports = pd.read_csv('./input/port_visits_train.csv')
meetings  = pd.read_csv('./input/meetings_train.csv')

In [6]:
labels.head()

,vessel_id,type
0,56d8509fe4b099a29d073a62,Container Vessel
1,56d950d9e4b0e60539c6b71d,Container Vessel
2,56d85a07e4b0fb60063969d8,Bulk Carrier
3,56d85736e4b030f1c0039905,Oil Tanker
4,56db31f9e4b02c227b748c14,Tug


In [12]:
vassels_types = labels['type']

In [18]:
vassels_types.value_counts()

Bulk Carrier        7404
Container Vessel    4057
Tug                 3419
Oil Tanker          2981
Fishing Vessel      2200
Passenger Vessel    1038
Reefer               543
Name: type, dtype: int64

In [23]:
types = vassels_types.unique()

In [30]:
types_dict = dict()
for i, t in enumerate(types):
    types_dict[t] = i

In [58]:
types_dict

{'Bulk Carrier': 1,
 'Container Vessel': 0,
 'Fishing Vessel': 5,
 'Oil Tanker': 2,
 'Passenger Vessel': 6,
 'Reefer': 4,
 'Tug': 3}

In [44]:
labels['num_types'] = labels['type'].apply(lambda x: types_dict[x])

In [50]:
labels.head()

,vessel_id,type,num_types
0,56d8509fe4b099a29d073a62,Container Vessel,0
1,56d950d9e4b0e60539c6b71d,Container Vessel,0
2,56d85a07e4b0fb60063969d8,Bulk Carrier,1
3,56d85736e4b030f1c0039905,Oil Tanker,2
4,56db31f9e4b02c227b748c14,Tug,3


In [35]:
ports.head()

,ves_id,start_time,duration_min,port_id,country,Lat,Long,port_name
0,56db7bf5e4b087b8d90a9d5e,2016-03-10 07:55:05,1641,57ce88b62e4e5fe35e7b3008,Indonesia,1.042,103.325,PT
1,56d88f20e4b0e60539c699ca,2016-03-31 07:37:35,730,5358fc78b68ca120a07dbca5,China,34.664,119.519,Lianyungang
2,56d96b7be4b0cdbda2acd645,2016-06-16 07:24:58,141,5358fc78b68ca120a07dbb33,Netherlands,51.689,4.590,Moerdijk
3,56d88f20e4b0e60539c699ca,2016-03-10 05:28:48,931,53720b4f57b2d3980edf9bfa,China,36.016,120.233,Qingdao
4,56d835ace4b09b379252ab21,2016-04-26 18:12:19,1,5358fc77b68ca120a07db099,France,47.276,-1.885,Cordemais


In [39]:
ports.columns = ['vessel_id', 'start_time', 'duration_min', 'port_id', 'country', 'Lat',
       'Long', 'port_name'] 

In [64]:
ports_full = pd.merge(ports, labels, left_on='vessel_id', right_on='vessel_id')

In [104]:
vessels_ports = ports_full.groupby(by='vessel_id')['port_id'].unique().reset_index()

In [127]:
vessels_ports['num_ports'] = vessels_ports['port_id'].apply(lambda x: len(x))

In [140]:
vessels_ports[vessels_ports.num_ports == 1]['port_id'].apply(lambda x: x[0]).unique().shape

(703,)

In [134]:
labels

,vessel_id,type,num_types
0,56d8509fe4b099a29d073a62,Container Vessel,0
1,56d950d9e4b0e60539c6b71d,Container Vessel,0
2,56d85a07e4b0fb60063969d8,Bulk Carrier,1
3,56d85736e4b030f1c0039905,Oil Tanker,2
4,56db31f9e4b02c227b748c14,Tug,3
5,56d839f0e4b002eabab58240,Bulk Carrier,1
6,56dcc15fe4b009004fbe1a23,Reefer,4
7,56d96d9ee4b0515e6629201a,Oil Tanker,2
8,56db8801e4b07967d3afcb96,Bulk Carrier,1
9,56d8367be4b0550004876e52,Fishing Vessel,5
